In [1]:
from pathlib import Path

data_dir = Path.cwd() / "data" / "uncompressed"

mimiic_dir = data_dir / "mimiciv" / "3.1"
eicu_dir = data_dir / "eicu-crd" / "2.0"

In [2]:
import pandas as pd
import dask

pd.options.mode.string_storage = "pyarrow"
dask.config.set({"dataframe.convert-string": True})

In [ ]:
import dask.dataframe as dd

from dask.distributed import Client

client = Client(n_workers=6)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 18,Total memory: 47.05 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42869,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:44637,Total threads: 3
Dashboard: http://127.0.0.1:46053/status,Memory: 7.84 GiB
Nanny: tcp://127.0.0.1:41917,


2025-09-02 21:02:16,673 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 2925f0ce80c676eef6d206813d7f590c initialized by task ('shuffle-transfer-2925f0ce80c676eef6d206813d7f590c', 9) executed on worker tcp://127.0.0.1:37463
2025-09-02 21:02:17,084 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle fa68d97c0fadbd1e703b360bb21225e9 initialized by task ('hash-join-transfer-fa68d97c0fadbd1e703b360bb21225e9', 9) executed on worker tcp://127.0.0.1:37463
2025-09-02 21:02:32,032 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 7b0b763fb39bdca5e0e8797abf7fc796 initialized by task ('hash-join-transfer-7b0b763fb39bdca5e0e8797abf7fc796', 9) executed on worker tcp://127.0.0.1:37463
2025-09-02 21:02:32,249 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 2925f0ce80c676eef6d206813d7f590c deactivated due to stimulus 'task-finished-1756846952.2340229'
2025-09-02 21:02:34,149 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 7b0b763fb39bdca5e0e8797abf7

In [4]:
from open_icu.config.utils import load_yaml_configs
from open_icu.config.source import SourceConfig

configs = load_yaml_configs(Path.cwd().parent / "configs" / "source", SourceConfig)
configs

[SourceConfig(name='mimic', version='3.1', tables=[TableConfig(name='medications', path='icu/inputevents.csv', fields=[FieldConfig(field='subject_id', type='int64'), FieldConfig(field='endtime', type='datetime'), FieldConfig(field='amount', type='float32'), FieldConfig(field='amountuom', type='string'), FieldConfig(field='starttime', type='datetime'), FieldConfig(field='rateuom', type='string'), FieldConfig(field='rate', type='float32'), FieldConfig(field='itemid', type='int64')], join=[JoinConfig(name='d_items', path='icu/d_items.csv', fields=[FieldConfig(field='itemid', type='int64'), FieldConfig(field='label', type='string')], both_on=['itemid'], left_on=[], right_on=[], how='left')], events=[EventConfig(name='dosage', fields=MEDSFieldsConfig(subject_id='subject_id', time='endtime', code=['label', 'amountuom'], numeric_value='amount', text_value=None)), EventConfig(name='rate', fields=MEDSFieldsConfig(subject_id='subject_id', time='starttime', code=['label', 'rateuom'], numeric_valu

In [5]:
from open_icu.meds.processor import process_table

for config in configs:
    for table in config.tables:
        process_table(table, mimiic_dir, Path.cwd() / "output")

2025-09-02 21:13:14,426 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('read-csv-1532163bd4f4100d2e41732a17794c1d', 360))" coro=<Worker.execute() done, defined at /workspaces/.venv/lib/python3.13/site-packages/distributed/worker_state_machine.py:3607>> ended with CancelledError
2025-09-02 21:13:14,429 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name='gather_dep(tcp://127.0.0.1:37463, {...})' coro=<Worker.gather_dep() done, defined at /workspaces/.venv/lib/python3.13/site-packages/distributed/worker_state_machine.py:3607>> ended with CancelledError
2025-09-02 21:13:14,573 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:37463 -> tcp://127.0.0.1:44637
Traceback (most recent call last):
  File "/workspaces/.venv/lib/python3.13/site-packages/tornado/iostream.py", line 962, in _handle_write
    num_bytes = self.write_to_fd(self._write_buffer.peek(size))
  File "/workspaces

KeyboardInterrupt: 

In [1]:
import dask.dataframe as dd
from pathlib import Path

df = dd.read_parquet(Path.cwd() / "output" / "data" / "medications_dosage_0.parquet").head()
df

,subject_id,time,numeric_value,text_value,code
0,10002013.0,2160-05-19 05:34:00,40.0,,Insulin - Glargine//units
1,10002114.0,2162-02-18 06:01:00,200.0,,Thiamine//mg
2,10002667.0,2187-02-25 05:06:00,10.0,,Insulin - Glargine//units
3,10002760.0,2141-04-21 05:19:00,30.0,,Insulin - Glargine//units
4,10002930.0,2196-04-15 08:59:00,100.0,,Thiamine//mg


In [ ]:
dd.read_parquet(Path.cwd() / "output" / "data" / "medications_rate_0.parquet").head()

,subject_id,time,numeric_value,text_value,code
0,10015931.0,2177-03-28 02:36:00,1.005025,,Aminocaproic acid (Amicar)//grams/hour
1,10015931.0,2177-03-26 10:45:00,1.003344,,Aminocaproic acid (Amicar)//grams/hour
2,10015931.0,2177-03-28 12:33:00,1.000000,,Aminocaproic acid (Amicar)//grams/hour
3,10015931.0,2177-03-27 16:40:00,1.006711,,Aminocaproic acid (Amicar)//grams/hour
4,10015931.0,2177-03-25 23:30:00,4.000000,,Aminocaproic acid (Amicar)//grams/hour
5,10015931.0,2177-03-26 00:32:00,1.000000,,Aminocaproic acid (Amicar)//grams/hour
6,10015931.0,2177-03-26 20:43:00,1.003344,,Aminocaproic acid (Amicar)//grams/hour
7,10015931.0,2177-03-27 06:41:00,1.000067,,Aminocaproic acid (Amicar)//grams/hour
8,10257607.0,2161-11-27 20:27:00,1.000000,,Aminocaproic acid (Amicar)//grams/hour
9,10516278.0,2164-12-14 21:00:00,0.504185,,Aminocaproic acid (Amicar)//grams/hour


In [3]:
dd.read_parquet(Path.cwd() / "output" / "data" / "chartevents_dosage_0.parquet").head()

,subject_id,time,numeric_value,text_value,code
0,10001884.0,2131-01-15 21:00:00,50.0,50,Permanent Pacemaker Rate///min
1,10001884.0,2131-01-16 00:00:00,50.0,50,Permanent Pacemaker Rate///min
2,10001884.0,2131-01-16 20:00:00,50.0,50,Permanent Pacemaker Rate///min
3,10001884.0,2131-01-17 00:00:00,50.0,50,Permanent Pacemaker Rate///min
4,10001884.0,2131-01-15 08:38:00,50.0,50,Permanent Pacemaker Rate///min


In [ ]:
df = dd.read_parquet(Path.cwd() / "output" / "metadata" / "codes.parquet").compute()
df

,code,description,parent_codes
